In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from trajectory_io import *
import torch 
from torch.autograd import Variable
import os
%matplotlib inline

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
def squeeze_out_pc(depth, extrinsic, intrinsic):
    assert isinstance(intrinsic, np.ndarray)
    assert intrinsic.shape == (3, 3)
    assert isinstance(depth, torch.autograd.Variable)
    assert len(depth.data.shape) == 2
    assert isinstance(extrinsic, np.ndarray)
    assert extrinsic.shape == (4, 4)
    
    h, w = depth.shape
    iy = np.arange(h)
    ix = np.arange(w)
    
    mesh = [torch.autograd.Variable(torch.from_numpy(a.astype(np.float32))) * depth for a in np.meshgrid(ix, iy)]
    image_cloud = torch.stack(mesh + [depth], dim=-1).reshape(-1, 3)
#     image_cloud = np.concatenate([image_cloud, np.ones([w*h, 1])], axis=-1)
    assert image_cloud.shape == (w * h, 3)
    
    pc_camera_coord = torch.from_numpy(np.linalg.inv(intrinsic).astype(np.float32)) @ image_cloud.T
    pc_global_coord = (torch.from_numpy(np.linalg.inv(extrinsic).astype(np.float32)) @ torch.cat([pc_camera_coord, torch.ones([1, w*h])], dim=0)).T
    pc_global_coord = pc_global_coord[:, :3]
    
    return pc_global_coord

def voxel_down_sample(pc, voxel_length):
    assert isinstance(pc, torch.autograd.Variable)
    assert len(pc.shape) == 2 and pc.shape[-1] == 3
    
    voxel_points = dict()
    
    for i in range(pc.shape[0]):
        point = pc[i]
        voxel = tuple(np.floor((point / voxel_length).numpy()))
        if voxel in voxel_points:
            voxel_points[voxel].append(point)
        else:
            voxel_points[voxel] = [point]
            
    downsampled_pc = []
    for v in voxel_points:
        downsampled_pc.append(torch.stack(voxel_points[v], dim=0).mean(dim=0))
    
    return torch.stack(downsampled_pc, dim=0)
      
def psi_discrete(nu, mu):
    if nu >= -mu:
        with torch.no_grad():
            sgn = torch.sign(nu)
        if 1 < nu/mu:
            return torch.autograd.Variable(torch.tensor(sgn)).double()
        else:
            return ((nu/mu) * sgn).double()
#         return min(1, nu/mu) * sgn
    else:
        return torch.autograd.Variable(torch.tensor(0)).double()
    
def psi_vector(nu, mu):
    zeros = torch.zeros(nu.data.shape[0])
    ones = torch.ones(nu.data.shape[0])
    print(nu.data.dtype)
    assert isinstance(nu, torch.autograd.Variable)
    assert len(nu.data.shape) == 1
    with torch.no_grad():
        outside = (nu >= -mu).float()
        sgn = torch.sign(nu)
        print(np.logical_and(nu.numpy() > -mu, nu.numpy() < 0).astype(np.float32).mean())
    zeros = torch.zeros(nu.data.shape[0])
    ones = torch.ones(nu.data.shape[0])
#     return outside * torch.min(ones, nu/mu) * sgn + (ones - outside) * zeros

    #It seems, that *sgn is error in the article
    return outside * torch.min(ones, nu/mu) + (ones - outside) * zeros




class DifferentialTSDFVolume():
    def __init__(self, voxel_length, mu, d, h, w, gf_corner_coord):
        # D, H, W, correspond to x, y, z axes in global frame 
        
        assert isinstance(gf_corner_coord, np.ndarray)
        assert gf_corner_coord.shape == (3, )
        
        self.voxel_length = voxel_length
        self.mu = mu
        self.weight = 0.0
        self.gf_corner_coord = gf_corner_coord
        self.h = h 
        self.w = w
        self.d = d
        self.nh = int(np.floor(h / voxel_length)) + 1
        self.nw = int(np.floor(w / voxel_length)) + 1
        self.nd = int(np.floor(d / voxel_length)) + 1
        self.volume = torch.autograd.Variable(torch.zeros([self.nd, self.nh, self.nw]))
        
        assert tuple(self.volume.data.shape) == (self.nd, self.nh, self.nw)
        
    def get_current_pointcloud(self, eps=0.2):
        return self.get_pointcloud(self.volume, eps)
    
    def get_pointcloud(self, tsdf_volume, eps=0.2):
        pc = []
        for i in range(tsdf_volume.data.shape[0]): # run along X axis
            for j in range(tsdf_volume.data.shape[1]): # run along Y axis
                for k in range(tsdf_volume.data.shape[2]): # run along Z axis
                    if tsdf_volume[i, j, k] != 0 and tsdf_volume[i, j, k] * self.mu < eps:
                        pc.append(np.array([self.gf_corner_coord[0] + i * self.voxel_length, 
                                            self.gf_corner_coord[1] + j * self.voxel_length, 
                                            self.gf_corner_coord[2] + k * self.voxel_length]))
        return np.stack(pc)
                        
        
    def individual_tsdf(self, depth, extrinsic, intrinsic):
        assert isinstance(depth, torch.autograd.Variable)
        assert len(tuple(depth.data.shape)) == 2
        assert depth.data.dtype == torch.float32
        
        assert isinstance(extrinsic, np.ndarray)
        assert extrinsic.shape == (4, 4)
        assert extrinsic.dtype == np.float32
        
        assert isinstance(intrinsic, np.ndarray)
        assert intrinsic.shape == (3, 3)
        assert intrinsic.dtype == np.float32
        
        tsdf = []
        
        #Coordinates of camera in the global frame
        
        t_g_k = (torch.from_numpy(np.linalg.inv(extrinsic)) @ torch.cat([torch.zeros([3]), torch.ones([1])]))[:3]
        
        active_pixels = {}
        debug_df = []
        for i in range(self.nd): # run along X axis
            for j in range(self.nh): # run along Y axis
                for k in range(self.nw): # run along Z axis
                    #Point coordinates in the global frame
                    
                    p = np.zeros([3])
                    p[0] = self.gf_corner_coord[0] + i * self.voxel_length
                    p[1] = self.gf_corner_coord[1] + j * self.voxel_length
                    p[2] = self.gf_corner_coord[2] + k * self.voxel_length
                    

                    
                    #Calculation projection to reference depth frame
                    x = (intrinsic @ (extrinsic @ np.concatenate([p, np.ones([1])])[:, None])[:3, :]).squeeze()
                    x = (x / x[2])[:2] 
                    x_floor = np.floor(x).astype(np.int32)

                
                    #Calculating lambda - depth scaling factor
                    lam  = np.linalg.norm(np.linalg.inv(intrinsic) @ np.concatenate([x, np.ones([1])])[:, None])
                                        
                    #Calculate truncated tsdf
                    if x[0] > 0.0 and x[0] < depth.data.shape[1] and x[1] > 0.0 and x[1] < depth.data.shape[0]:
#                         print(type(depth[x_floor[1], x_floor[0]]))
#                         print(type(psi_discrete(float(np.linalg.norm(t_g_k - p)/lam) - depth[x_floor[1], x_floor[0]],
#                                                      self.mu)))
                        tsdf.append(psi_discrete(float(np.linalg.norm(t_g_k - p)/lam) - depth[x_floor[1], x_floor[0]],
                                                     self.mu))
                        debug_df.append(float(np.linalg.norm(t_g_k - p)/lam) - depth[x_floor[1], x_floor[0]])
                    else:
                        tsdf.append(torch.autograd.Variable(torch.tensor(0)).double())
    
#                         tsdf[i, j, k] += psi_discrete(float(np.linalg.norm(t_g_k - p)/lam) - depth[x_floor[1], x_floor[0]],
#                                                     self.mu)
        print(len(tsdf))
        return torch.stack(tsdf).reshape([self.nd, self.nh, self.nw])
    
    def integrate(self, depth, extrinsic, intrinsic, w):
        #Here we assume that weight does not depend on the point
        new_tsdf = self.individual_tsdf(depth, extrinsic, intrinsic)
        
        self.volume = (self.weight * self.volume + w * new_tsdf) / (self.weight + w)
        self.weight += w
        


class DifferentialVectorTSDFVolume():
    def __init__(self, voxel_length, mu, d, h, w, gf_corner_coord):
        # D, H, W, correspond to x, y, z axes in global frame 
        
        assert isinstance(gf_corner_coord, np.ndarray)
        assert gf_corner_coord.shape == (3, )
        
        self.voxel_length = voxel_length
        self.mu = mu
        self.weight = 0.0
        self.gf_corner_coord = gf_corner_coord
        self.h = h 
        self.w = w
        self.d = d
        self.nh = int(np.floor(h / voxel_length)) + 1
        self.nw = int(np.floor(w / voxel_length)) + 1
        self.nd = int(np.floor(d / voxel_length)) + 1
        self.volume = torch.autograd.Variable(torch.zeros([self.nd, self.nh, self.nw])).reshape(-1)
        print(self.volume.data.shape)
        assert tuple(self.volume.data.shape) == (self.nd * self.nh * self.nw, )
        
    def get_current_pointcloud(self, eps=0.2):
        return self.get_pointcloud(self.volume, eps)
    
    def get_pointcloud(self, tsdf_volume, eps=0.2):
        tsdf_volume = tsdf_volume.data.numpy()
        
        flag = np.logical_and(np.logical_and(np.abs(tsdf_volume) > 0.0001, tsdf_volume < 0.0), 
                                             tsdf_volume * self.mu < eps).astype(np.int32)
        ind = np.nonzero(flag)[0]
        
        p = np.meshgrid(np.linspace(0, (self.nd-1)*self.voxel_length, num=self.nd),
                np.linspace(0, (self.nh-1)*self.voxel_length, num=self.nh), 
                np.linspace(0, (self.nw-1)*self.voxel_length, num=self.nw), 
                indexing='ij')
        p = np.stack(p, axis=-1)
        p = p.reshape(-1, 3) + self.gf_corner_coord[None, :]   
        
        return p[ind]
        
#         pc = []
#         for i in range(tsdf_volume.data.shape[0]): # run along X axis
#             for j in range(tsdf_volume.data.shape[1]): # run along Y axis
#                 for k in range(tsdf_volume.data.shape[2]): # run along Z axis
#                     if tsdf_volume[i, j, k] != 0 and tsdf_volume[i, j, k] * self.mu < eps:
#                         pc.append(np.array([self.gf_corner_coord[0] + i * self.voxel_length, 
#                                             self.gf_corner_coord[1] + j * self.voxel_length, 
#                                             self.gf_corner_coord[2] + k * self.voxel_length]))
#         return np.stack(pc)
                        
        
    def individual_tsdf(self, depth, extrinsic, intrinsic):
        assert isinstance(depth, torch.autograd.Variable)
        assert len(tuple(depth.data.shape)) == 2
        assert depth.data.dtype == torch.float32
        
        assert isinstance(extrinsic, np.ndarray)
        assert extrinsic.shape == (4, 4)
        assert extrinsic.dtype == np.float32
        
        assert isinstance(intrinsic, np.ndarray)
        assert intrinsic.shape == (3, 3)
        assert intrinsic.dtype == np.float32
        
        tsdf = torch.autograd.Variable(torch.zeros([self.nd, self.nh, self.nw])).reshape(-1)
        
        #Coordinates of camera in the global frame
        
        t_g_k = (torch.from_numpy(np.linalg.inv(extrinsic)) @ torch.cat([torch.zeros([3]), torch.ones([1])]))[:3]
        
        active_pixels = {}
        
        #Creating pointcloud, corresponding to voxel_grid
        p = np.meshgrid(np.linspace(0, (self.nd - 1)*self.voxel_length, num=self.nd),
                        np.linspace(0, (self.nh - 1)*self.voxel_length, num=self.nh), 
                        np.linspace(0, (self.nw - 1)*self.voxel_length, num=self.nw), 
                        indexing='ij')
        p = np.stack(p, axis=-1)
        assert p.shape == (self.nd, self.nh, self.nw, 3)
        p = p.reshape(-1, 3) + self.gf_corner_coord[None, :]
        assert p.shape == (self.nd * self.nh * self.nw, 3)

        pc_size = p.shape[0]
        
        #Calculation projection to reference depth frame
        x = intrinsic @ (extrinsic @ np.concatenate([p, np.ones([pc_size, 1])], axis=1).T)[:3, :]
        x = (x / x[2:3, :])[:2, :].T
        assert x.shape == (pc_size, 2)
        
        x_floor = np.floor(x).astype(np.int32)
        
        #Calculating lambda - depth scaling factor
        lam = np.linalg.norm(np.linalg.inv(intrinsic) @ np.concatenate([x, np.ones([pc_size, 1])], axis=1).T, axis=0)
        assert lam.shape == (pc_size, )
        
        #Filter out points, which do not intersect with reference frame
        ray_intersect = np.logical_and(np.logical_and(x[:, 0] > 0, x[:, 0] < depth.data.shape[1]), 
                                       np.logical_and(x[:, 1] > 0, x[:, 1] < depth.data.shape[0]))
        assert ray_intersect.shape == (pc_size, )
        ray_intersect = np.nonzero(ray_intersect.astype(np.int32))[0].astype(np.int32)
        
        active_x = x[ray_intersect]
        active_p = p[ray_intersect]
        active_lam = lam[ray_intersect]
        active_x_floor = x_floor[ray_intersect]
        active_depth = depth[active_x_floor[:, 1], active_x_floor[:, 0]]
        
        debug_df = torch.from_numpy(np.linalg.norm(t_g_k.reshape(1, 3) - active_p, axis=1)/active_lam).float() - active_depth
#         active_tsdf = psi_vector(torch.from_numpy(np.linalg.norm(t_g_k.reshape(1, 3) - active_p, axis=1)/active_lam).float() - active_depth, self.mu)
        active_tsdf = psi_vector(active_depth - torch.from_numpy(np.linalg.norm(t_g_k.reshape(1, 3) - active_p, axis=1)/active_lam).float(), self.mu)


        tsdf[ray_intersect] = active_tsdf
        return tsdf
  
    def integrate(self, depth, extrinsic, intrinsic, w):
        #Here we assume that weight does not depend on the point
        new_tsdf = self.individual_tsdf(depth, extrinsic, intrinsic)
        
        self.volume = (self.weight * self.volume + w * new_tsdf) / (self.weight + w)
        self.weight += w
        
                    
        

In [4]:
def voxel_down_sample(pc, voxel_length):
    assert isinstance(pc, torch.autograd.Variable)
    assert len(pc.shape) == 2 and pc.shape[-1] == 3
    
    voxel_points = dict()
    
    for i in range(pc.shape[0]):
        point = pc[i]
        voxel = tuple(np.floor((point / voxel_length).numpy()))
        if voxel in voxel_points:
            voxel_points[voxel].append(point)
        else:
            voxel_points[voxel] = [point]
            
    downsampled_pc = []
    for v in voxel_points:
        downsampled_pc.append(torch.stack(voxel_points[v], dim=0).mean(dim=0))
    
    return torch.stack(downsampled_pc, dim=0)
            
    
            

In [2]:
K = np.array([[2.666666666666666970e+03, 0.000000000000000000e+00, 9.600000000000000000e+02],
     [0.000000000000000000e+00, 2.666666666666666970e+03, 5.400000000000000000e+02],
     [0.000000000000000000e+00, 0.000000000000000000e+00, 1.000000000000000000e+00]])

In [5]:
npy = np.load('blender_render_depth4/{:01d}.npy'.format(0)).squeeze()
extrinsic = np.loadtxt('blender_render_depth4/RT_%01d.txt' % 0).astype(np.float32)
# extrinsic = np.identity(4).astype(np.float32)
intrinsic = K.astype(np.float32)

x = np.array([npy.shape[1]//2, npy.shape[0]//2, 1.0])
p =  np.linalg.inv(extrinsic) @ np.concatenate([np.linalg.inv(K) @ (x * npy[npy.shape[0]//2, npy.shape[1]//2]), np.ones([1])])
p = p[:3]

In [7]:
# volume = DifferentialTSDFVolume(voxel_length=0.1, mu=0.3, d=4.0, h=4.0, w=4.0, gf_corner_coord=np.array([-2, -4, -2]))
vector_volume = DifferentialVectorTSDFVolume(voxel_length=0.03, mu=0.1, d=4.0, h=4.0, w=4.0, gf_corner_coord=np.array([-2, -4, -2]))
input_depth = []
individual_tsdf = []
# [5, 120, 276, 170, 50, 220]
for i in [0, 180, 220]:
# for i in range(0, 220, 10):
    print("Integrating {:d} frame".format(i))
    npy = np.load('blender_render_depth4/{:01d}.npy'.format(i)).squeeze()
    extrinsic = np.loadtxt('blender_render_depth4/RT_%01d.txt' % i).astype(np.float32)
    # extrinsic = np.identity(4).astype(np.float32)
    intrinsic = K.astype(np.float32) 
    input_depth.append(torch.autograd.Variable(torch.from_numpy(npy)))
    individual_tsdf.append(vector_volume.individual_tsdf(input_depth[-1], extrinsic, intrinsic))
    
#     volume.integrate(input_depth[-1], extrinsic, intrinsic, w=0.1)
    vector_volume.integrate(input_depth[-1], extrinsic, intrinsic, w=0.1)

torch.Size([2406104])
Integrating 0 frame
torch.float32
0.0022204972
torch.float32
0.0022204972
Integrating 180 frame
torch.float32
0.0025327825
torch.float32
0.0025327825
Integrating 220 frame
torch.float32
0.0019197317
torch.float32
0.0019197317


In [8]:
pc = o3d.geometry.PointCloud()
# pc.points = o3d.utility.Vector3dVector(vector_volume.get_pointcloud((individual_tsdf[0] + individual_tsdf[1])/2, eps=1.0))
pc.points = o3d.utility.Vector3dVector(vector_volume.get_current_pointcloud(eps=0.8))
# pc_0.paint_uniform_color(color=np.array([[1, 0, 0]]).T)
o3d.visualization.draw_geometries([pc])